In [ ]:
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings, ChatHuggingFace
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled
from dotenv import load_dotenv

/Users/abhayas/Desktop/machine learning/genAI/langChainENV/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
load_dotenv()

True

## Indexing

In [25]:
video_id = "Cbqtxys2qPM" #id only
try:
    #language na rakhe best language return garxa
    transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    
except TranscriptsDisabled:
    print("No captions available for this video.")
    
    

In [26]:
# print(transcript_list)

In [27]:
transcript = " ".join(snippet.text for snippet in transcript_list.snippets)

In [28]:
transcript

"Hey, heat. Hey. It's been exactly one year since Donald Trump took office. He was inaugurated on this day in 2025, January 20th. Everyone knew it was going to be chaotic. After all, this was an experienced and vengeful Trump. He wouldn't spare anyone, not allies, not adversaries. But I'm sure no one expected this level of drama. One year into his presidency, the most pressing issue is not Ukraine or Iran. It is Greenland. Trump is adamant about taking over this island. He wants Europe and Denmark to serve it on a platter. And if they don't, Trump is threatening tariffs starting in February. At first, Europe criticized this move, but now they want talks. They are trying to reason with the US president. First, the NATO chief reached out. He sent a message to Donald Trump. A message that Trump leaked on social media. Listen to this. Dear Donald, I will use my media engagements in Davos to highlight your work in Syria, in Gaza, and in Ukraine. I am committed to finding a way forward in Gr

In [29]:
# transcript_text = "\n".join(
#     f"[{s.start:.2f}s] {s.text}"
#     for s in transcript_list.snippets
# )
# print(transcript_text)

text splitting

In [37]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 350, chunk_overlap = 90)
chunks = splitter.create_documents([transcript])

In [38]:
len(chunks)

25

In [39]:
for chunk in chunks:
    print(chunk)
    print()
    print()
    print()

page_content='Hey, heat. Hey. It's been exactly one year since Donald Trump took office. He was inaugurated on this day in 2025, January 20th. Everyone knew it was going to be chaotic. After all, this was an experienced and vengeful Trump. He wouldn't spare anyone, not allies, not adversaries. But I'm sure no one expected this level of drama. One year into his'



page_content='not adversaries. But I'm sure no one expected this level of drama. One year into his presidency, the most pressing issue is not Ukraine or Iran. It is Greenland. Trump is adamant about taking over this island. He wants Europe and Denmark to serve it on a platter. And if they don't, Trump is threatening tariffs starting in February. At first,'



page_content='platter. And if they don't, Trump is threatening tariffs starting in February. At first, Europe criticized this move, but now they want talks. They are trying to reason with the US president. First, the NATO chief reached out. He sent a message to Donald Tr

In [40]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(chunks, embeddings)

In [41]:
vector_store.index_to_docstore_id

{0: '8b560d2b-c13f-413b-a2ac-047a8064d467',
 1: '67782484-ac78-458f-ad62-256adcb045d7',
 2: 'be4f04cd-3a31-49cc-bc21-51aaaec84d5d',
 3: '7e35b7af-f055-4c26-a24d-03bb0feec3b1',
 4: '7e4841e4-5e8b-4d7d-8bfb-6a5ca61da7ff',
 5: 'e3ead64c-145f-4600-ae23-b07fc2d9e06d',
 6: '26c76a47-c284-47d9-b991-44311aecf0f9',
 7: '25c822ec-e388-4711-b255-8c5bcf878d14',
 8: 'bd5a6e3f-6293-4d73-8ca0-1b7910111b4a',
 9: 'c6988b67-d789-4acc-8307-130d247f8881',
 10: '9a6b262d-e8da-4026-8753-378fec109952',
 11: '52b87c32-ebf4-49e6-bcf2-d8e02ba06a90',
 12: '6b9641ef-67d9-4175-94bc-6167cc05b7b6',
 13: '35965fbd-bdc8-44b5-bd98-ae527016f322',
 14: 'c65967fa-d8e6-4bc1-9199-abb2921b6919',
 15: 'b3b392e4-93b4-4623-bc25-10fdc9370fda',
 16: '371a54d2-a165-4d8a-a2fe-41d314083b19',
 17: '363281a0-32e8-4153-8dda-1d860e10f08a',
 18: '39adfd6c-d5dd-415a-96b5-032737398743',
 19: '9a20ea8f-773b-4619-9caf-ad53cf668546',
 20: 'c2dadb31-7666-4680-9c0c-c8d56a8c870d',
 21: 'a707223b-fd54-4390-9190-feba0a173dd3',
 22: '9421db03-2025-

Retrieval

In [47]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x137a4c1a0>, search_kwargs={'k': 3})

In [48]:
retriever.invoke('possibility of war')

[Document(id='363281a0-32e8-4153-8dda-1d860e10f08a', metadata={}, page_content='on the war in Ukraine. They should not be focusing on Greenland. So military coercion is very much possible. The other option is economic coercion. Trump has threatened to impose 10% tariffs on eight European countries. Many of them are weighing a strong response. But the US Treasury Secretary has warned against it. He wants Europe to let things'),
 Document(id='9421db03-2025-4b5d-bf96-4b41b3c4d7f5', metadata={}, page_content="operation, it has no risk. The Danes are not about to fight the US military, so Greenland is basically a sitting duck. Let's see if diplomacy can bring things back from the brink. >> The world doesn't fit in boxes, so why should your news? From battlefields to negotiation tables, from internet outrage to real world injustice, >> the conversations"),
 Document(id='67782484-ac78-458f-ad62-256adcb045d7', metadata={}, page_content="not adversaries. But I'm sure no one expected this level 

Augmentation

In [53]:
prompt = PromptTemplate(
    template="""
    You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
      """,
      input_variables=['context', 'question']
)

In [73]:
question = "is trump wanting greenland"
retrieved_docs = retriever.invoke(question)


In [74]:
retrieved_docs

[Document(id='b3b392e4-93b4-4623-bc25-10fdc9370fda', metadata={}, page_content="is that's none of his business. Greenland's prime minister has rejected Trump's moves. Greenland's people have rejected him. Denmark has rejected him. So, the diplomatic option is off the table. Europe will not offer Greenland willingly, which leaves the other option, coercion. Will Trump use American military force to capture Greenland? He was"),
 Document(id='c2dadb31-7666-4680-9c0c-c8d56a8c870d', metadata={}, page_content="security. First principle, full solidarity with Greenland and the Kingdom of Denmark. The sovereignity and integrity of their territory is nonnegotiable. So all eyes are are on Davos. Though frankly, don't get your hopes up. Trump has gone too far to give up on Greenland and Europe cannot be seen as surrendering their territory. It's a perfect"),
 Document(id='c65967fa-d8e6-4bc1-9199-abb2921b6919', metadata={}, page_content="have a right of ownership anyway? There are no written docume

In [75]:
context_text = "\n\n".join(docs.page_content for docs in retrieved_docs)
context_text

"is that's none of his business. Greenland's prime minister has rejected Trump's moves. Greenland's people have rejected him. Denmark has rejected him. So, the diplomatic option is off the table. Europe will not offer Greenland willingly, which leaves the other option, coercion. Will Trump use American military force to capture Greenland? He was\n\nsecurity. First principle, full solidarity with Greenland and the Kingdom of Denmark. The sovereignity and integrity of their territory is nonnegotiable. So all eyes are are on Davos. Though frankly, don't get your hopes up. Trump has gone too far to give up on Greenland and Europe cannot be seen as surrendering their territory. It's a perfect\n\nhave a right of ownership anyway? There are no written documents. It's only that a boat landed there hundreds of years ago, but we had boats landing there also. Clearly, Trump is building a case. He's questioning Denmark's rights over Greenland. The problem is that's none of his business. Greenland'

In [76]:
final_prompt = prompt.invoke({"context": context_text, "question": question})
final_prompt

StringPromptValue(text="\n    You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      is that's none of his business. Greenland's prime minister has rejected Trump's moves. Greenland's people have rejected him. Denmark has rejected him. So, the diplomatic option is off the table. Europe will not offer Greenland willingly, which leaves the other option, coercion. Will Trump use American military force to capture Greenland? He was\n\nsecurity. First principle, full solidarity with Greenland and the Kingdom of Denmark. The sovereignity and integrity of their territory is nonnegotiable. So all eyes are are on Davos. Though frankly, don't get your hopes up. Trump has gone too far to give up on Greenland and Europe cannot be seen as surrendering their territory. It's a perfect\n\nhave a right of ownership anyway? There are no written documents. It's only that a boat landed there hundreds of

output generation

In [77]:
llm = HuggingFaceEndpoint(
    repo_id="openai/gpt-oss-20b",
    task ="text-generation",
    # temperature= 2
)
model = ChatHuggingFace(llm = llm)

In [78]:
result = model.invoke(final_prompt.text)


In [79]:
result

AIMessage(content='Yes – the transcript indicates that Trump has been pursuing Greenland, with his moves being rejected by Greenland’s prime minister and Denmark.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 424, 'prompt_tokens': 318, 'total_tokens': 742}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3023a70d60', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bdcab-c929-7ba0-9631-137515da7fa4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 318, 'output_tokens': 424, 'total_tokens': 742})